In [ ]:
import pandas as pd
import ipcalc
import parse_functions as pf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import user_agents as ua
from helpers import filter_other, autopct_format
import calendar
import janitor
import pytz

In [ ]:
# Daten aus CSV laden
filename = "data/acces_log.csv"
access_log = pd.read_csv(filename)

In [ ]:
city_ip4 = pd.read_csv('data/geo-city/GeoLite2-City-Blocks-IPv4.csv')
city_ip4.head()
locations = pd.read_csv('data/geo-city/GeoLite2-City-Locations-de.csv')
locations.head()

In [ ]:
access_log['ip_n'] = pf.ip_to_int(access_log['ip'])
access_log.head()

In [ ]:
city_ip4['ip_range'] = city_ip4['network'].map(lambda network: ipcalc.Network(network))
city_ip4['ip_first'] = city_ip4['ip_range'].map(lambda ip: int(ip[0])).astype('int32')
city_ip4['ip_last'] = city_ip4['ip_range'].map(lambda ip: int(ip[-1])).astype('int32')

city_ip4.head()

In [ ]:
ips = access_log['ip_n'].unique()
ip_map = pd.DataFrame(ips, columns=['ip_n'])
ip_map = ip_map.conditional_join(city_ip4, ('ip_n', 'ip_first', '>='), ('ip_n', 'ip_last', '<='))

In [ ]:
# Zwischenstand Als CSV speichern
filename = "data/ip_mapping.csv"
ip_map.to_csv(filename, index=False)

In [ ]:
# Laden aus CSV (dauert keine 6 Minuten)
filename = "data/ip_mapping.csv"
ip_map = pd.read_csv(filename)
ip_map.head()

In [ ]:
ip_map['geoname_id'] = ip_map['geoname_id'].astype('Int64')
locations['geoname_id'] = locations['geoname_id'].astype('Int64')
ip_locations = ip_map.merge(locations, how='left', on='geoname_id', suffixes=(None, '_i'))
ip_locations.head()

In [ ]:
access_log['ip_n'] = access_log['ip_n'].astype('int32')
ip_locations['ip_n'] = ip_locations['ip_n'].astype('int32')
log_with_locations = access_log.merge(ip_locations, how='left', on='ip_n', suffixes=(None, '_i'))
log_with_locations.head()

In [ ]:
country_counts = log_with_locations['country_name'].value_counts()

country_counts_filtered = filter_other(country_counts, 0.01)

country_counts_filtered.plot(kind='pie', title='Countries', ylabel='Country', autopct=autopct_format(country_counts_filtered))